# Image Classification Challenge

In [28]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
from torchvision.io import read_image
import torch
from torch import nn as nn
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import torch.utils as utils
import os

### **LOAD DATA**

I created a dataset class to load the images from local directories and used Dataloader to load the training and testing set. Note here that to align with the structure of the dataset classes, I used ```product id``` of each testing image as the ```label```.

Note that the dataset ```5_shot``` is in the ```PATH``` folder.

In [2]:
CATEGORY = range(22)
# PATH = '/Users/zhilinzhou/Desktop/'
TRAIN_PATH = PATH + '5_shot/train/'
TEST_PATH = PATH + '5_shot/test'
transform_pipe = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size=(224, 224)),  
    transforms.ToTensor()
])

class myDataset(Dataset):
    def __init__(self, dir, train=False, categories=None, transform=None):
        self.dir, self.labels = [], []
        self.size = 0
        if train:
            for c in categories:
                path = dir + str(c)
                os.chdir(path)
                for file in os.listdir():
                    if file.endswith('.jpg'):
                        self.dir.append(f"{path}/{file}")
                        self.labels.append(c)
                        self.size += 1
        else:
            os.chdir(dir)
            for file in os.listdir():
                if file.endswith('.jpg'):
                    self.dir.append(f"{dir}/{file}")
                    self.labels.append(file[:-4])
                    self.size += 1
        self.transform = transform
    
    def __getitem__(self, idx):
        image = read_image(self.dir[idx])
        image = self.transform(image)
        label = self.labels[idx]
        return image, label
    
    def __len__(self):
        return self.size

In [41]:
TRAIN_DATA = myDataset(dir=TRAIN_PATH, train=True, categories=CATEGORY, transform=transform_pipe)
TEST_DATA = myDataset(dir=TEST_PATH, transform=transform_pipe)
TRAIN_LOADER = utils.data.DataLoader(
    TRAIN_DATA,
    batch_size=8
)
TEST_LOADER = utils.data.DataLoader(
    TEST_DATA,
    batch_size=8
)

print(f"{TRAIN_DATA.size} training data, {TEST_DATA.size} test data.")

110 training data, 517 test data.


### **MODEL**

In [63]:
MODEL = torchvision.models.resnet50(pretrained=True)
MODEL

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### **TRAINING**

In [59]:
class myModel():
    def __init__(self, model, train_loader, test_loader, device, optimizer, num_epoch=15):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.num_epoch = num_epoch
        ## optimizer?
        if optimizer == 'Adam':
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=2e-4)
        elif optimizer == 'AdamW':
            self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-4, weight_decay=0.1)
        else:
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.128, momentum=0.875, weight_decay=3)
        self.criterion = torch.nn.CrossEntropyLoss()
    
    def train(self):
        print("Start training...")
        self.model.train()
        total_loss, total_accuracy = [], []
        n = len(self.train_loader.dataset)
        for epoch in range(1,self.num_epoch+1):
            loss_, accuracy_ = 0, 0
            samples = 0
            for i, batch in enumerate(self.train_loader, 0):
                inputs, labels = batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                self.optimizer.zero_grad()

                result = self.model(inputs)
                loss = self.criterion(result, labels)

                loss.backward()
                self.optimizer.step()

                loss_ += loss.item()
                correct = np.sum(np.argmax(result.detach().numpy(), axis=1) == labels.detach().numpy())
                accuracy_ += correct
                samples += inputs.shape[0]

                if i % 10 == 0:
                    print("Epoch: %d\t [%d/%d - %d%%]: loss - %.4f \t accuracy - %.2f%%" \
                            % (epoch, samples, n, samples/n*100, loss/samples, accuracy_/samples*100))
            total_loss.append(loss_ / samples)
            total_accuracy.append(accuracy_ / samples)
            print("==> Epoch %d: loss - %.4f, accuracy - %.2f%%" \
                    % (epoch, total_loss[-1], total_accuracy[-1]*100))
        print("Training finished!\n")
        return total_loss, total_accuracy
    
    def test(self):
        print("Start testing...")
        self.model.eval()
        ids = []
        predictions = []
        with torch.no_grad():
            for i, batch in enumerate(self.test_loader, 0):
                inputs, labels = batch
                inputs = inputs.to(device)
                for id_ in labels:
                    ids.append(id_)
                pred = self.model(inputs)
                predictions.append(np.argmax(pred.detach().numpy(), axis=1))
        print("Testing finished!\n")
        return ids, predictions

**PRESENT RESULTS**

In [64]:
# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

model = myModel(MODEL, TRAIN_LOADER, TEST_LOADER, device, 'AdamW', num_epoch=20)
loss, accuracy = model.train()

Start training...
Epoch: 1	 [8/110 - 7%]: loss - 1.0844 	 accuracy - 0.00%
Epoch: 1	 [88/110 - 80%]: loss - 0.0853 	 accuracy - 0.00%
==> Epoch 1: loss - 0.9977, accuracy - 0.00%
Epoch: 2	 [8/110 - 7%]: loss - 0.1865 	 accuracy - 87.50%
Epoch: 2	 [88/110 - 80%]: loss - 0.0279 	 accuracy - 54.55%
==> Epoch 2: loss - 0.3214, accuracy - 45.45%
Epoch: 3	 [8/110 - 7%]: loss - 0.0165 	 accuracy - 100.00%
Epoch: 3	 [88/110 - 80%]: loss - 0.0064 	 accuracy - 92.05%
==> Epoch 3: loss - 0.0964, accuracy - 89.09%
Epoch: 4	 [8/110 - 7%]: loss - 0.0217 	 accuracy - 100.00%
Epoch: 4	 [88/110 - 80%]: loss - 0.0038 	 accuracy - 96.59%
==> Epoch 4: loss - 0.0356, accuracy - 97.27%
Epoch: 5	 [8/110 - 7%]: loss - 0.0106 	 accuracy - 100.00%
Epoch: 5	 [88/110 - 80%]: loss - 0.0015 	 accuracy - 100.00%
==> Epoch 5: loss - 0.0157, accuracy - 100.00%
Epoch: 6	 [8/110 - 7%]: loss - 0.0046 	 accuracy - 100.00%
Epoch: 6	 [88/110 - 80%]: loss - 0.0004 	 accuracy - 100.00%
==> Epoch 6: loss - 0.0069, accuracy - 1

### **OUTPUT PREDICTIONS**

In [65]:
id, predictions = model.test()
submissions = pd.DataFrame({
    'id': id,
    'category': np.concatenate(predictions).reshape(-1,).astype("int")
}).set_index('id')
submissions.head(10)

Start testing...
Testing finished!



,category
id,
63,3
189,7
77,3
162,6
176,7
88,3
348,17
360,15
406,17


In [67]:
os.chdir(PATH)
submissions.to_csv("final_submission.csv")